In [1]:
# parse URL's from html 3
from bs4 import BeautifulSoup
import requests
import urllib
import time

## Parse all the info that we need

In [2]:
def title (soup):
    # We get the title of the movie
    title = soup.select("#firstHeading")[0].text
    return title 

In [3]:
def intro(soup):
    # In this part we get the INTRO of the movie
    try:
        sec = soup.findAll('p')[0]
        if sec == soup.find("p", class_="mw-empty-elt"):
            section_intro = soup.findAll('p')[1]
        else:
            section_intro = sec
        nextNode = section_intro
        intro = []
        intro.append(nextNode.text)

        while True:
            nextNode = nextNode.find_next_sibling()
            if nextNode and nextNode.name == 'p':
                intro.append(nextNode.text)

            else:
                break          
        intro_s = ""

        for ele in intro: 
            intro_s += ele
            
        return intro_s
    
    except IndexError:
        intro_s = None
        return intro_s

In [4]:
def plot (soup):
    try:    
        # In this part we get the PLOT of the movie
        sec = soup.findAll('h2')[0]
        if sec.text == 'Contents' or sec.text == 'Cast':
            section_plot = soup.findAll('h2')[1]
            if section_plot.text == 'Cast':
                section_plot = soup.findAll('h2')[1]
        else:
            section_plot = sec
        nextNode = section_plot.find_next_sibling('p')

        plot = []
        while True:

            if nextNode and nextNode.name == 'p':
                plot.append(nextNode.text)
                nextNode = nextNode.find_next_sibling()
            else:
                break          
        plot_s = ""

        for ele in plot: 
            plot_s += ele
        return plot_s
    except IndexError:
        plot_s = None
        return plot_s

## Infobox

In [5]:
def infobox(soup):
    try:
        table = soup.find('table', class_='infobox vevent')
        nextNode = table
        table2 = table.find_all('tr')

        dic={}
        for th in table2[1:]:
            if th.find('th'):            
                dic[th.find('th').text] = th.find('td').get_text(strip=True, separator='|').split('|')

        standard_dic = {
        "Directed by" : "",
        "Produced by": "",
        "Written by": "",
        "Starring": "",
        "Music by": "", 
        "Release date": "",
        "Running time": "",
        "Country": "",
        "Language": "",
        "Budget": ""} 

        # In this part we check if the keys of the infobox are the same as the ones requested   
        shared_items = {k: dic[k] for k in dic.keys() & standard_dic.keys()}



        # We transform the list into strings
        for k, v in shared_items.items():
            shared_items[k] = ", ".join(v)

        # Difference, we would like to find the missing INFO of this movie
        value = { k : standard_dic[k] for k in set(standard_dic) - set(dic) }

        # Replace missing INFO with NaN
        value = {k: None if not v else v for k, v in value.items() }


        # Let's combine these two dictionaries
        final = dict(shared_items, **value)

        return final
    
    except AttributeError:
        final = {
        "Directed by" : None,
        "Produced by": None,
        "Written by": None,
        "Starring": None,
        "Music by": None, 
        "Release date": None,
        "Running time": None,
        "Country": None,
        "Language": None,
        "Budget": None} 
        
        
        return final

    
    

## Write the info into a TSV file

In [6]:
import glob
import os.path
import csv

dir_path = r"C:\Users\Luca\Desktop\-\Università\Magistrale\Primo anno\Primo semestre\ADM\Homeworks\Homework #3\articles"

c = 0   
for file_name in glob.glob(os.path.join(dir_path, "*.html")):
    
    with open(file_name, encoding="utf8") as html_file:
        soup = BeautifulSoup(html_file)
        t = title(soup)
        i = intro(soup)
        p = plot(soup)
        
        
        # Write TSV file for each movie, we create a unique dictionary
        dic_title = {'Title' : t}
        dic_intro = {'Intro' : i}
        dic_plot  = {'Plot' : p}
        dic_infobox = infobox(soup)
        
        temp = dict(dic_title, **dic_intro) 
        temp2 = dict(temp, **dic_plot)
        final = dict(temp2, **dic_infobox) # it's the unique dictionary we were talking of before
        
        with open(r'TSV\article_{}.tsv'.format(c), 'wt', encoding="utf8") as out_file:
            tsv_writer = csv.DictWriter(out_file, final.keys(), delimiter ='\t')
            tsv_writer.writeheader()
            tsv_writer.writerow(final)
    c += 1